## Finding Magnetic Nulls near a reconnection event observed by MMS

This notebook will show how to use PySPEDAS to locate and characterize magnetic null points.

During a dayside magneopause crossing on 2023-03-24, MMS observed the signature of a reconnection event: the magnetic field direction reversed (so, nearly antiparallel field lines in close proximity), simultaneous with fast (~400 km/sec) particle flows that also reversed direction during the event.

Magnetic null points are an important aspect of the reconnection process.  Since MMS has four probes taking simultaneous magnetic field measurements in a tight, approximately tetrahedral formation, we have enough information to model the magnetic field in 3 dimensions in the vicinity of the MMS constellation.

The method here is known as the First Order Taylor Expansion (FOTE) method.   By using the MMS field measurements, we can estimate the B field vector at the center of the tetrahedron. We can also estimate the 3x3 Jacobian matrix, capturing the partial derivative of the vector components Bx, By, and Bz with respect to the local X, Y, and Z coordinates.   

Together, these two estimates can be used to solve for a location where Bx = By = Bz: that is, a magnetic null point.   As long as the field measurements aren't identical between probes, a null can be found.  Whether that finding is credible or not depends on how well the assumption of linear field gradients holds over the distence to the null, and on some other figures of merit that can be derived from the trace and eigenvalues of the Jacobean matrix.

We'll start by importing some pyspedas and pytplot tools.

In [ ]:
import os
os.environ["SPEDAS_DATA_DIR"] = "/home/jovyan/scratch_space/pyspedas_data"


In [ ]:
# Here are some imports we'll need below.
import pyspedas
import pytplot

# We'll import a few pytplot routines, to simplify things below
from pytplot import timespan, highlight, time_double, options, tplot, tnames


We need to know the MMS spacecraft locations, so we'll get them with mms_load_state()

In [ ]:
state_vars = pyspedas.mms.mms_load_state(trange=['2023-03-24/07:00','2023-03-24/08:00'],probe=['1','2','3','4'])

# For FGM, we'll use the survey data for the overview plot, and burst data for the
# null analysis.

fgm_vars = pyspedas.mms.fgm(trange=['2023-03-24/07:00','2023-03-24/08:00'],probe=['1','2','3','4'],data_rate='brst',time_clip=True)
fgm_vars = pyspedas.mms.fgm(trange=['2023-03-24/07:00','2023-03-24/08:00'],probe=['1'],data_rate='srvy',time_clip=True)

fpi_vars = pyspedas.mms.mms_load_fpi(trange=['2023-03-24/07:00','2023-03-24/08:00'],probe=['1','2','3','4'],time_clip=True,data_rate='fast')

# Reset time limits from previous examples
timespan('2023-03-24/07:00',60.0,'minutes')

pytplot.tplot_names()

The MMS ephemeris data is in ECI coordinates.  We'll be working in GSE, so we need to transform the loaded variables.

In [ ]:
# To use the MMS qcotrans routine, the quaternions representing the transforms must be
# loaded from the MEC dataset.
pyspedas.mms.mec(trange=['2023-03-24/07:00','2023-03-24/08:00'], probe=['1','2','3','4'])

for i in ('1','2','3','4'): 
    pyspedas.mms.mms_qcotrans('mms'+i+'_defeph_pos','mms'+i+'_defeph_pos_gse',in_coord='eci',out_coord='gse')
    pyspedas.mms.mms_qcotrans('mms'+i+'_defeph_vel','mms'+i+'_defeph_vel_gse',in_coord='eci',out_coord='gse')

We'll generate a summary plot of the data loaded so far:

In [ ]:

timespan('2023-03-24/07:00',10,'minutes')
options('mms1_des_numberdensity_fast','ylog',False)
fig1_vars=['mms1_defeph_pos_gse','mms1_defeph_vel_gse','mms1_fgm_b_gse_srvy_l2','mms1_dis_bulkv_dbcs_fast','mms1_des_numberdensity_fast','mms1_des_energyspectr_omni_fast','mms1_dis_energyspectr_omni_fast']

tplot(fig1_vars)

Let's get a sense of the probe separations and orbital velocity during this time:

In [ ]:
from pytplot import subtract
from pyspedas import tvectot

subtract('mms1_defeph_pos_gse','mms2_defeph_pos_gse',newname='mms1_mms2_dist')
subtract('mms1_defeph_pos_gse','mms3_defeph_pos_gse',newname='mms1_mms3_dist')
subtract('mms1_defeph_pos_gse','mms4_defeph_pos_gse',newname='mms1_mms4_dist')
subtract('mms2_defeph_pos_gse','mms3_defeph_pos_gse',newname='mms2_mms3_dist')
tvectot(['mms1_mms2_dist','mms1_mms3_dist','mms1_mms4_dist','mms2_mms3_dist','mms1_defeph_vel_gse'])
tplot(['mms1_mms2_dist_tot','mms1_mms3_dist_tot','mms1_mms4_dist_tot', 'mms2_mms3_dist_tot','mms1_defeph_vel_gse_tot'])



So for future reference, the tetrahedron size is on the order of 75 km, and the constellation is moving at about 3.45 km/sec.

Now we will find the magnetic nulls during this event, and examine the field toplology for a couple of them:

In [ ]:
mms_positions = ['mms1_defeph_pos_gse','mms2_defeph_pos_gse','mms3_defeph_pos_gse','mms4_defeph_pos_gse']
mms_fields = ['mms1_fgm_b_gse_brst_l2','mms2_fgm_b_gse_brst_l2','mms3_fgm_b_gse_brst_l2','mms4_fgm_b_gse_brst_l2']

null_vars = pyspedas.find_magnetic_nulls_fote(fields=mms_fields, positions=mms_positions, smooth_fields=True)

pytplot.tplot_names()

print(null_vars)


The variables returned by find_magnetic_nulls_fote are:

null_pos: The position of reported null points, in the same coordinate system as the position inputs (here, GSE)

null_bary_dist:  The scalar distance between the null and the consteallation barycenter

null_bary_dist_types:  A composite tplot variable with a scatter plot of null topology types, superimposed on the null_bary_dist values

null_sc_distances: The scalar distances between the null point and each of the four spacecraft.

null_fom:  Two scalar figures of merit, eta and xi, representing the confidence in the null detection and null topology classification

null_typecode: Integer type codes representing the topological classification of the null points.

max_reconstruction_error:  The maximum difference, in nT, between the spacecraft magnetic field measurements, and the Taylor expansion of the field using the calculated field value and field gradients at the constellation barycenter.

It's hard to make sense of the plots of magnetic null results on timescales longer than a few seconds, so we'll want to zoom in on some shorter time intervals.,

First, we'll look at 60 seconds worth of data starting at 07:01:


In [ ]:
timespan('2023-03-24/07:01:00',60,'seconds')
options('null_fom','yrange',[0.0,0.5])
tplot(['mms1_fgm_b_gse_brst_l2','mms2_fgm_b_gse_brst_l2','mms3_fgm_b_gse_brst_l2','mms4_fgm_b_gse_brst_l2','null_sc_distances','null_bary_dist_types','null_fom'])

Just after 07:01:40, all four spacecraft are seeing low fields, so let's zoom in on that:

In [ ]:
timespan('2023-03-24/07:01:40',10,'seconds')
options('null_fom','yrange',[0.0,1.0])
options('null_sc_distances','yrange',[0.0, 100.0])
options('null_bary_dist','yrange',[0.0,100.0])
tplot(['mms1_fgm_b_gse_brst_l2','mms2_fgm_b_gse_brst_l2','mms3_fgm_b_gse_brst_l2','mms4_fgm_b_gse_brst_l2','null_sc_distances','null_bary_dist','null_bary_dist_types','null_fom'])

Looking at the quality variables (lower=better), and null_sc and null_barycenter distances, it appears that several magnetic nulls were reliably detected and classified, perhaps even enclosed by the MMS tetrahedron, around 41-42 and 46 seconds after 07:01 UT.   


Now we'll look at another time interval starting at 07:07:00:

In [ ]:
timespan('2023-03-24/07:07:00',60,'seconds')
options('null_fom','yrange',[0.0,0.5])
options('null_sc_distances','yrange',[0.0, 1000.0])
options('null_bary_dist','yrange',[0.0,1000.0])
tplot(['null_sc_distances','null_bary_dist_types','null_fom'])

Now we'll look at the time interval 07:07:20 - 07:07:25:

In [ ]:
timespan('2023-03-24/07:07:20',5,'seconds')
tplot(['null_sc_distances','null_bary_dist_types','null_fom'])

Between 21-15 seconds after 7:07, we seem to have a reliable detection of a Type A radial null at a distance of about 500 km from the formation.  This coincides with the magnetic reconnection signature observed in MMS-1 at this time: anti-parallel magnetic field lines, fast ion flows and flow reversals.